Section 1: Imports and Data Structures

This section imports necessary libraries and defines the Process class, which serves as a blueprint for each process to be scheduled.

In [ ]:
# Import the 'sys' module to interact with the Python interpreter.
import sys
# Import 'files' from 'google.colab' to enable file uploads in Colab.
from google.colab import files
# Import 'io' to work with in-memory I/O streams for file content.
import io

# Define the Process class to hold process attributes.
class Process:
    # The constructor initializes a new Process object.
    def __init__(self, arrival_time, burst_time, pid):
        # The time the process arrives in the system.
        self.arrival_time = arrival_time
        # The total CPU time required for the process.
        self.burst_time = burst_time
        # The remaining execution time. Initially, it's the same as burst_time.
        self.remaining_time = burst_time
        # The time at which the process completes execution.
        self.completion_time = 0
        # The total time from arrival to completion.
        self.turnaround_time = 0
        # The total time the process waits in the ready queue.
        self.waiting_time = 0
        # The time from arrival until the first time the process is executed. -1 indicates it hasn't run yet.
        self.response_time = -1
        # The process ID, used for tie-breaking in some algorithms.
        self.pid = pid

Section 2: Metric Calculation

This section contains a helper function to compute the average turnaround, response, and waiting times for a list of processes after a scheduling algorithm has been run.

In [ ]:
# Function to calculate average metrics for a set of processes.
def calculate_metrics(processes):
    # Get the number of processes in the list.
    num_processes = len(processes)
    # Avoid division by zero if there are no processes.
    if num_processes == 0:
        return 0.0, 0.0, 0.0

    # Calculate the total sum of each metric.
    total_turnaround_time = sum(p.turnaround_time for p in processes)
    total_response_time = sum(p.response_time for p in processes)
    total_waiting_time = sum(p.waiting_time for p in processes)

    # Calculate the average of each metric.
    avg_turnaround_time = total_turnaround_time / num_processes
    avg_response_time = total_response_time / num_processes
    avg_waiting_time = total_waiting_time / num_processes

    # Return the average metrics as a tuple.
    return avg_turnaround_time, avg_response_time, avg_waiting_time

Section 3: Scheduling Algorithms

This section implements the three required CPU scheduling algorithms: First-Come, First-Served (FCFS), Shortest Job First (SJF), and Round Robin (RR).

In [ ]:
# Implements the FCFS (First-Come, First-Served) scheduling algorithm.
# FCFS: Processes are executed in the order they arrive.
def fcfs(processes_data):
    # Create a list of Process objects from the input data.
    processes = [Process(p[0], p[1], idx) for idx, p in enumerate(processes_data)]
    if not processes:
        return 0.0, 0.0, 0.0
    # Sort processes by arrival time.
    processes.sort(key=lambda x: x.arrival_time)

    # Initialize the current simulation time.
    current_time = 0
    # Iterate through the sorted processes.
    for p in processes:
        # If the CPU is idle, advance the time to the process's arrival time.
        if current_time < p.arrival_time:
            current_time = p.arrival_time

        # Calculate the metrics for the current process.
        p.response_time = current_time - p.arrival_time
        p.completion_time = current_time + p.burst_time
        p.turnaround_time = p.completion_time - p.arrival_time
        p.waiting_time = p.turnaround_time - p.burst_time
        # Update the current time to when this process finishes.
        current_time = p.completion_time
    
    return calculate_metrics(processes)

# Implements the non-preemptive SJF (Shortest Job First) scheduling algorithm.
# SJF: The process with the shortest burst time is executed to completion.
def sjf(processes_data):
    processes = [Process(p[0], p[1], idx) for idx, p in enumerate(processes_data)]
    if not processes:
        return 0.0, 0.0, 0.0

    current_time = 0
    completed_processes = 0
    n = len(processes)
    
    # Ready queue for processes that have arrived.
    ready_queue = []
    # List of processes that have not yet arrived, sorted by arrival time.
    unarrived_processes = sorted(processes, key=lambda x: x.arrival_time)
    unarrived_idx = 0

    # Simulate until all processes are complete.
    while completed_processes < n:
        # Add processes that have arrived by `current_time` to the ready queue.
        while unarrived_idx < n and unarrived_processes[unarrived_idx].arrival_time <= current_time:
            ready_queue.append(unarrived_processes[unarrived_idx])
            unarrived_idx += 1
        
        # If the ready queue is empty, the CPU is idle.
        if not ready_queue:
            # If there are still processes to arrive, advance time to the next arrival.
            if unarrived_idx < n:
                current_time = unarrived_processes[unarrived_idx].arrival_time
                continue
            # All processes have arrived and the queue is empty, so simulation ends.
            else:
                break

        # Sort the ready queue by the shortest burst time.
        # Tie-breaking: FCFS (by arrival time), then by PID.
        ready_queue.sort(key=lambda x: (x.burst_time, x.arrival_time, x.pid))
        
        # Select the process with the shortest burst time.
        shortest_job = ready_queue.pop(0)

        # Set the response time if it's the first time the process is executed.
        if shortest_job.response_time == -1:
            shortest_job.response_time = current_time - shortest_job.arrival_time

        # Execute the process to completion (non-preemptive).
        current_time += shortest_job.burst_time
        shortest_job.completion_time = current_time
        
        # Calculate process metrics.
        shortest_job.turnaround_time = shortest_job.completion_time - shortest_job.arrival_time
        shortest_job.waiting_time = shortest_job.turnaround_time - shortest_job.burst_time
        
        completed_processes += 1
    
    processes.sort(key=lambda x: x.pid)
    return calculate_metrics(processes)

# Implements the RR (Round Robin) scheduling algorithm with a quantum of 2[cite: 8].
def rr(processes_data, quantum=2):
    processes = [Process(p[0], p[1], idx) for idx, p in enumerate(processes_data)]
    if not processes:
        return 0.0, 0.0, 0.0
    n = len(processes)
    current_time = 0
    completed_processes = 0
    
    # Ready queue implemented as a list (FIFO behavior).
    q = []
    # List of unarrived processes, sorted by arrival time.
    unarrived_processes = sorted(processes, key=lambda x: x.arrival_time)
    unarrived_idx = 0
    # Set to track PIDs currently in the queue to prevent duplicates.
    in_queue_pids = set()

    # Set the initial simulation time to the arrival time of the first process.
    if unarrived_processes:
        current_time = unarrived_processes[0].arrival_time
    else:
        return 0.0, 0.0, 0.0

    while completed_processes < n:
        # Add processes that have arrived by `current_time` to the ready queue.
        while unarrived_idx < n and unarrived_processes[unarrived_idx].arrival_time <= current_time:
            p = unarrived_processes[unarrived_idx]
            if p.pid not in in_queue_pids:
                q.append(p)
                in_queue_pids.add(p.pid)
            unarrived_idx += 1
        
        # If the ready queue is empty, advance the time to the next arrival.
        if not q:
            if unarrived_idx < n:
                current_time = unarrived_processes[unarrived_idx].arrival_time
                continue
            else:
                break

        # Get the next process from the queue (FIFO).
        p = q.pop(0)
        in_queue_pids.remove(p.pid)

        # Set the response time if it's the first execution.
        if p.response_time == -1:
            p.response_time = current_time - p.arrival_time

        # Determine the execution time slice (quantum or remaining time).
        execute_time = min(quantum, p.remaining_time)
        current_time += execute_time
        p.remaining_time -= execute_time

        # After execution, add any new processes that arrived during this time slice.
        while unarrived_idx < n and unarrived_processes[unarrived_idx].arrival_time <= current_time:
            new_p = unarrived_processes[unarrived_idx]
            if new_p.pid not in in_queue_pids:
                q.append(new_p)
                in_queue_pids.add(new_p.pid)
            unarrived_idx += 1
        
        # If the process is not finished, put it back at the end of the queue.
        if p.remaining_time > 0:
            if p.pid not in in_queue_pids:
                q.append(p)
                in_queue_pids.add(p.pid)
        # If the process is finished, calculate its final metrics.
        else:
            p.completion_time = current_time
            p.turnaround_time = p.completion_time - p.arrival_time
            p.waiting_time = p.turnaround_time - p.burst_time
            completed_processes += 1

    processes.sort(key=lambda x: x.pid)
    return calculate_metrics(processes)